#  Deployment and inference

In [ ]:
import os
import sys

current_dir = os.getcwd()
kit_dir =  os.path.abspath(os.path.join(current_dir, '..'))
repo_dir = os.path.abspath(os.path.join(kit_dir, '..'))
sys.path.append(repo_dir)

from utils.dedicated_env.snsdk_wrapper import SnsdkWrapper

## Step by Step / Manual setting

First instantiate the SambaStudio client 

In [2]:
sambastudio_client = SnsdkWrapper()

2024-11-25 16:13:38,830 [INFO] Using variables from Snapi config to set up Snsdk.


### Create Project

#### Set Project configs

In [ ]:
project = {
    'project_name': 'byoc-fine-tuning-project',
    'project_description': 'this project will be used to test the BYOC and Fine-tuning e2e pipeline implementation'
}

In [8]:
# Execute the create project method from client with project parameters
sambastudio_client.create_project(
    project_name = project['project_name'],
    project_description = project['project_description']
)

2024-11-25 16:10:34,332 [INFO] Project with name 'byoc fine-tuning project' found with id b11867e6-7ca8-45bd-b09b-41cbc7ba73ce
2024-11-25 16:10:34,332 [INFO] Project with name 'byoc fine-tuning project' already exists with id 'b11867e6-7ca8-45bd-b09b-41cbc7ba73ce', using it


'b11867e6-7ca8-45bd-b09b-41cbc7ba73ce'

### Create a Bundle model

#### Set bundle model config 

In [4]:
composite_model= {
    'model_name':  'Suzume-Llama-3-8B-Multilingual-Bundle',
    'description': 'Bundle including base and public health fine-tuned Suzume-Llama-3-8B-Multilingual',
    'rdu_required': 8,
    'model_version': '1',
    'model_list': [
        'Suzume-Llama-3-8B-Multilingual',
        'Suzume-Llama-3-8B-Multilingual-Publichealth'
    ]
}

In [11]:
# Execute the create composite model method from client with composite model parameters
sambastudio_client.create_composite_model(       
        model_name = composite_model['model_name'],
        description = composite_model['description'],
        model_list = composite_model['model_list'],
        rdu_required = composite_model['rdu_required']
)

2024-11-25 16:10:57,701 [INFO] Model with name 'Suzume-Llama-3-8B-Multilingual-Bundle' not found
2024-11-25 16:10:57,971 [INFO] Model with name 'Suzume-Llama-3-8B-Multilingual' found with id b2bb7c02-3b18-4ef4-bd58-1b19669bce15
2024-11-25 16:10:58,214 [INFO] Model with name 'Suzume-Llama-3-8B-Multilingual-Publichealth' found with id c867b392-2d02-453d-9fd8-e14016e39153
2024-11-25 16:10:58,214 [INFO] Models to include in composite found with ids '[('Suzume-Llama-3-8B-Multilingual', 'b2bb7c02-3b18-4ef4-bd58-1b19669bce15'), ('Suzume-Llama-3-8B-Multilingual-Publichealth', 'c867b392-2d02-453d-9fd8-e14016e39153')]
2024-11-25 16:10:58,815 [INFO] Composite model with name Suzume-Llama-3-8B-Multilingual-Bundle created with id 47a71ddb-04d9-459f-9314-198e6f78458d


'47a71ddb-04d9-459f-9314-198e6f78458d'

### Create Endpoint

#### Set endpoint config 

In [5]:
endpoint = {
  'endpoint_name': 'suzume-multilingual-endpoint',
  'endpoint_description': 'endpoint for suzume multilingual bundle',
  'endpoint_instances': 1,
  'hyperparams': {}
}

In [6]:
# Execute the create endpoint method from client with endpoint parameters
sambastudio_client.create_endpoint(
    project_name=project['project_name'],
    endpoint_name=endpoint['endpoint_name'],
    endpoint_description=endpoint['endpoint_description'],
    model_name=composite_model['model_name'],
    model_version=composite_model['model_version'],
    instances=endpoint['endpoint_instances'],
    hyperparams=endpoint['hyperparams'],
    rdu_arch="SN40L-8",
)

2024-11-25 16:13:52,787 [INFO] Project with name 'byoc fine-tuning project' found with id b11867e6-7ca8-45bd-b09b-41cbc7ba73ce
2024-11-25 16:13:53,119 [INFO] Model with name 'Suzume-Llama-3-8B-Multilingual-Bundle' found with id 47a71ddb-04d9-459f-9314-198e6f78458d
2024-11-25 16:13:53,335 [INFO] Endpoint with name 'suzume-multilingual-endpoint' not found in project 'b11867e6-7ca8-45bd-b09b-41cbc7ba73ce'
2024-11-25 16:13:53,836 [INFO] Endpoint 'suzume-multilingual-endpoint' created


'b035b70f-a96a-49ed-82c1-50f2d3502f76'

In [ ]:
#get endpoint details, including api key and envs
sambastudio_client.get_endpoint_details(
    project_name=project['project_name'],
    endpoint_name=endpoint['endpoint_name']
)

### Inference

#### Get endpoint details

In [10]:
# store envs
endpoint_env = sambastudio_client.get_endpoint_details(
    project_name=project['project_name'],
    endpoint_name=endpoint['endpoint_name']
    )['langchain_wrapper_env']

2024-11-25 16:15:01,078 [INFO] Project with name 'byoc fine-tuning project' found with id b11867e6-7ca8-45bd-b09b-41cbc7ba73ce


In [ ]:
# instantiate langchain chat models to test inference 

from langchain_sambanova import ChatSambaNova

multilingual_llm = ChatSambaNova(
    sambanova_api_base=endpoint_env.get("SAMBANOVA_API_BASE"),
    sambastudio_api_key=endpoint_env.get("SAMBANOVA_API_KEY"),
    temperature = 0.01,
    max_tokens = 1024,
    top_p = 0.1,
    do_sample = False,
    process_prompt = False,
    model = "Suzume-Llama-3-8B-Multilingual"
)

multilingual_health_llm = ChatSambaNova(
    sambanova_api_base=endpoint_env.get("SAMBANOVA_API_BASE"),
    sambanova_api_key=endpoint_env.get("SAMBANOVA_API_KEY"),
    temperature = 0.01,
    max_tokens = 1024,
    top_p = 0.1,
    do_sample = False,
    process_prompt = False,
    model = "Suzume-Llama-3-8B-Multilingual-Publichealth"
)

In [ ]:
multilingual_llm.invoke("which are the best habits to improve my mental capabilities")

In [ ]:
multilingual_health_llm.invoke("Why im felling mental fatigue, can be covid related")

## Streamlined Execution

The bundle model creation and endpoint deployment can be done in a streamlined way setting all the composite model and endpoint parameters in a config file like in the [deploy_config.yaml](../deploy_config.yaml) example, and executing:

In [ ]:
config_file = os.path.join(kit_dir, 'deploy_config.yaml')
sambastudio_client = SnsdkWrapper(config_file)
sambastudio_client.create_project()
sambastudio_client.create_composite_model()
sambastudio_client.create_endpoint()
endpoint_env = sambastudio_client.get_endpoint_details()